## 1. Import Libraries & Datasets

### Install Necessary Libraries

In [1]:
# Install Tensorflow python library

pip install TensorFlow

SyntaxError: invalid syntax (3122256462.py, line 3)

### Import Libraries

In [2]:
# Import necessary python libraries & ignore futurewarnings

import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

### Import Datasets

In [3]:
# Create path

path = r"C:\Users\18478\OneDrive\Desktop\Career Foundry Data Analytics Course\Career Foundry Machine Learning"

In [4]:
# Load in unscaled weather dataset

climate_df = pd.read_csv(os.path.join(path, 'Data Sets', 'Dataset-weather-prediction-dataset-processed.csv'), index_col = False)

In [5]:
# Load in pleasant weather prediction answers dataset

pleasant_df =  pd.read_csv(os.path.join(path, 'Data Sets', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'), index_col = False)

In [6]:
# check output

climate_df.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [7]:
# Check shape

climate_df.shape

(22950, 170)

In [8]:
pleasant_df.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
pleasant_df.shape

(22950, 16)

## 2. Data Prep

### Check and Address Stations with Missing Data 

In [10]:
# Remove weather stations not included in "pleasant weather" answers
climate_df = climate_df.drop(['GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
                        'ROMA_cloud_cover', 'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 'ROMA_temp_mean',
                        'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min', 'TOURS_temp_max'], axis=1)

In [11]:
# Define the  observations

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [12]:
# Create a dictionary to store the count of stations for each observation type

station_counts = {}

for obs in observation_types:

    columns = [col for col in climate_df.columns if col.endswith(obs)]
    
# Count the number of stations for the  observation types
    
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type

print("Number of stations collecting each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")


Number of stations collecting each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


In [13]:
# Create a list of columns containing 'wind_speed' or 'snow_depth'

cols_to_drop = [col for col in climate_df.columns if '_wind_speed' in col or '_snow_depth' in col]

# Drop the columns
climate_df = climate_df.drop(cols_to_drop, axis=1)

In [14]:
climate_df.shape

(22950, 134)

In [15]:
# Find the stations with the above entries missing
# Get all column names
all_columns = climate_df.columns.tolist()
# Exclude 'DATE' and 'MONTH' columns
all_columns = [col for col in all_columns if col not in ['DATE', 'MONTH']]  
# Extract unique weather station names
weather_stations = set() 
for col in all_columns:
    station_name = col.split('_')[0] 
    weather_stations.add(station_name)

# Print the list of weather stations
print(weather_stations)

{'DUSSELDORF', 'BASEL', 'BELGRADE', 'DEBILT', 'BUDAPEST', 'MUNCHENB', 'MAASTRICHT', 'STOCKHOLM', 'MADRID', 'HEATHROW', 'LJUBLJANA', 'VALENTIA', 'OSLO', 'SONNBLICK', 'KASSEL'}


In [16]:
# Find stations missing observation types

observation_types = ['cloud_cover', 'humidity', 'pressure']

missing_stations_by_observation = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in climate_df.columns if col.endswith(obs)]
    
    # Extract station names by removing the observation type from the column names
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    
    # Identify stations that are in all_stations but missing from the current observation type
    missing_stations = weather_stations - station_names
    
    # Store the missing station names in the dictionary
    missing_stations_by_observation[obs] = missing_stations

# Print the missing station names for each observation type
for obs, missing_stations in missing_stations_by_observation.items():
    print(f"\nStations missing from {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Stations missing from cloud_cover:
KASSEL

Stations missing from humidity:
STOCKHOLM

Stations missing from pressure:
MUNCHENB


In [17]:
# Insert new columns into "unscaled" at specific positions.
# The data for these new columns is taken from weather stations they are close to

climate_df.insert(56,'KASSEL_cloud_cover', climate_df['DUSSELDORF_cloud_cover'])
climate_df.insert(119, 'STOCKHOLM_humidity', climate_df['OSLO_humidity'])
climate_df.insert(94,'MUNCHENB_pressure',climate_df['BASEL_pressure'])

In [18]:
climate_df.columns.tolist()

['DATE',
 'MONTH',
 'BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSS

In [19]:
# Check output

climate_df.shape

(22950, 137)

In [20]:
climate_df.to_csv(os.path.join(path, 'Data Sets', 'Unsupervised', 'weather_dataset_unscaled_prepared_with_date.csv'), index=False)

### Drop unnecssary columns 

In [22]:
# Drop temporal columns

climate_df.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [23]:
# Check output

climate_df.shape

(22950, 135)

In [27]:
pleasant_df.shape

(22950, 16)

In [28]:
pleasant_df.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
# drop date column from prediction answers dataset
pleasant_df.drop(columns = 'DATE', inplace = True)

In [30]:
pleasant_df.shape

(22950, 15)

### Save Prepared Data to File

In [25]:
# Export cleaned dataset as csv

climate_df.to_csv(os.path.join(path, 'Data Sets', 'Unsupervised', 'weather_dataset_unscaled_prepared.csv'), index=False)

## 3. Split and Reshape Data for Deep Learning

### Define X and Y shapes

In [31]:
# Creat an 'X' matrix by reloading and naming our data 'X'

X = pd.read_csv(os.path.join(path, 'Data Sets', 'Unsupervised', 'weather_dataset_unscaled_prepared.csv'), index_col=False)

In [32]:
y = pleasant_df

In [33]:
X.shape

(22950, 135)

In [34]:
y.shape

(22950, 15)

### Re-shape X Observations

In [35]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)

X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [36]:
# Reshape for 3D object

X = X.reshape(-1,15,9)

In [38]:
# check output

X.shape

(22950, 15, 9)

### Split X and Y Sets

In [40]:
# split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [41]:
# Check output

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


## 4. CNN Keras Model 

In [42]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [43]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,831 (11.06 KB)

 Trainable params: 2,831 (11.06 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1036 - loss: 4913.2847
Epoch 2/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1165 - loss: 48205.0977
Epoch 3/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1229 - loss: 167035.7500
Epoch 4/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1217 - loss: 351870.6562
Epoch 5/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1249 - loss: 619171.5625
Epoch 6/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1275 - loss: 987411.1250
Epoch 7/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1290 - loss: 1457690.0000
Epoch 8/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1304 - loss: 2027528.1250
Epoch 9/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1297 - loss: 2682928.5000
Epoch 10/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1303 - loss: 3453012.2500
Epoch 11/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1322 - loss: 4408781.0000
Epoch 12/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1315 - loss: 5485632.5000
Epoch 13/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1296 - loss: 6747850.0000
Epoc

In [46]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [47]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [48]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BELGRADE  DEBILT  DUSSELDORF  HEATHROW  KASSEL  LJUBLJANA  \
True                                                                           
BASEL           2       548       1         853        17      41         30   
BELGRADE        0       111       0         129         0       0          1   
BUDAPEST        0         8       0          61         0       0          1   
DEBILT          0         0       0          27         0       0          0   
DUSSELDORF      0         1       0           9         0       0          0   
HEATHROW        0         2       0          34         0       0          0   
KASSEL          0         1       0           1         0       0          0   
LJUBLJANA       0         2       0          14         0       0          3   
MAASTRICHT      0         0       0           5         0       0          1   
MADRID          0        14       0         144         5       0          2   

In [49]:
epochs = 30
batch_size = 16
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [50]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1329 - loss: 988.6993
Epoch 2/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1339 - loss: 9389.5410
Epoch 3/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1256 - loss: 33229.8398
Epoch 4/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1152 - loss: 75029.1953
Epoch 5/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1129 - loss: 132721.7031
Epoch 6/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1074 - loss: 208205.7656
Epoch 7/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1072 - loss: 297062.2500
Epoch 8/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1128 - loss: 382373.8438
Epoch 9/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1077 - loss: 487000.5625
Epoch 10/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1084 - loss: 617778.6875
Epoch 11/30
1076/1076 - 4s - 4ms/step - accuracy: 0.1102 - loss: 761963.1250
Epoch 12/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1136 - loss: 923708.4375
Epoch 13/30
1076/1076 - 4s - 3ms/step - accuracy: 0.1144 - loss: 1084972.3750
Epoch 14/30
10

In [53]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [54]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BELGRADE  BUDAPEST  DUSSELDORF  HEATHROW  MADRID  MUNCHENB  OSLO
True                                                                        
BASEL             39         1          74      3456      90        13     9
BELGRADE           7         0           0      1085       0         0     0
BUDAPEST           2         0           0       212       0         0     0
DEBILT             0         0           0        82       0         0     0
DUSSELDORF         0         0           0        29       0         0     0
HEATHROW           0         0           0        82       0         0     0
KASSEL             0         0           0        11       0         0     0
LJUBLJANA          0         0           0        61       0         0     0
MAASTRICHT         0         0           0         9       0         0     0
MADRID             0         0           0       454       4         0     0
MUNCHENB           0         0     

In [55]:
epochs = 30
batch_size = 16
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [56]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 14, 16)         │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,191 (24.18 KB)

 Trainable params: 6,191 (24.18 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0706 - loss: 23.7801
Epoch 2/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1576 - loss: 22.2749
Epoch 3/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1668 - loss: 27.3470
Epoch 4/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1663 - loss: 27.3470
Epoch 5/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1658 - loss: 27.3835
Epoch 6/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1662 - loss: 27.3882
Epoch 7/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1662 - loss: 27.3910
Epoch 8/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1666 - loss: 27.3910
Epoch 9/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1666 - loss: 27.3938
Epoch 10/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1666 - loss: 27.3938
Epoch 11/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1665 - loss: 27.3938
Epoch 12/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1665 - loss: 27.3938
Epoch 13/30
1076/1076 - 5s - 4ms/step - accuracy: 0.1665 - loss: 27.3938
Epoch 14/30
1076/1076 - 5s - 5ms/step - accuracy: 0.1665 - l

In [59]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [60]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BELGRADE  DUSSELDORF  KASSEL  MAASTRICHT  MADRID  STOCKHOLM
True                                                                   
BASEL           1385         779     783           1     731          3
BELGRADE         886          65     136           0       5          0
BUDAPEST         158          32      24           0       0          0
DEBILT            76           4       2           0       0          0
DUSSELDORF        23           2       4           0       0          0
HEATHROW          47          25      10           0       0          0
KASSEL             8           2       1           0       0          0
LJUBLJANA         22          33       6           0       0          0
MAASTRICHT         4           0       5           0       0          0
MADRID           104         197     119           0      38          0
MUNCHENB           0           3       5           0       0          0
OSLO               1   

In [61]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [62]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 6s - 6ms/step - accuracy: 0.0575 - loss: 22.9788
Epoch 2/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0485 - loss: 22.3220
Epoch 3/30
1076/1076 - 9s - 8ms/step - accuracy: 0.0450 - loss: 23.0572
Epoch 4/30
1076/1076 - 6s - 5ms/step - accuracy: 0.0381 - loss: 26.2364
Epoch 5/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 6/30
1076/1076 - 5s - 4ms/step - accuracy: 0.0348 - loss: 18.4836
Epoch 7/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0348 - loss: 18.4835
Epoch 8/30
1076/1076 - 6s - 5ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 9/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 10/30
1076/1076 - 5s - 4ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 11/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 12/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 13/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0348 - loss: 18.4798
Epoch 14/30
1076/1076 - 5s - 5ms/step - accuracy: 0.0348 - l

In [65]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [66]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BUDAPEST  DEBILT  MAASTRICHT  MUNCHENB  OSLO  VALENTIA
True                                                              
BASEL           2859     207         157       418     1        40
BELGRADE        1087       4           0         1     0         0
BUDAPEST         214       0           0         0     0         0
DEBILT            82       0           0         0     0         0
DUSSELDORF        29       0           0         0     0         0
HEATHROW          82       0           0         0     0         0
KASSEL            11       0           0         0     0         0
LJUBLJANA         61       0           0         0     0         0
MAASTRICHT         9       0           0         0     0         0
MADRID           458       0           0         0     0         0
MUNCHENB           8       0           0         0     0         0
OSLO               5       0           0         0     0         0
STOCKHOLM          4 

In [67]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [68]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 7s - 7ms/step - accuracy: 0.6199 - loss: 8839.8633
Epoch 2/30
1076/1076 - 5s - 5ms/step - accuracy: 0.6433 - loss: 95302.4844
Epoch 3/30
1076/1076 - 5s - 4ms/step - accuracy: 0.6435 - loss: 321522.0625
Epoch 4/30
1076/1076 - 5s - 4ms/step - accuracy: 0.6434 - loss: 703853.8750
Epoch 5/30
1076/1076 - 5s - 4ms/step - accuracy: 0.6434 - loss: 1245639.0000
Epoch 6/30
1076/1076 - 5s - 5ms/step - accuracy: 0.6435 - loss: 1978530.0000
Epoch 7/30
1076/1076 - 4s - 4ms/step - accuracy: 0.6436 - loss: 2875678.2500
Epoch 8/30
1076/1076 - 5s - 5ms/step - accuracy: 0.6436 - loss: 4065033.0000
Epoch 9/30
1076/1076 - 5s - 5ms/step - accuracy: 0.6437 - loss: 5521079.0000
Epoch 10/30
1076/1076 - 4s - 4ms/step - accuracy: 0.6437 - loss: 7163379.0000
Epoch 11/30
1076/1076 - 5s - 4ms/step - accuracy: 0.6437 - loss: 9066295.0000
Epoch 12/30
1076/1076 - 5s - 5ms/step - accuracy: 0.6437 - loss: 11265240.0000
Epoch 13/30
1076/1076 - 5s - 4ms/step - accuracy: 0.6437 - loss: 13766791.0000


In [71]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [72]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  VALENTIA
True                       
BASEL        3679         3
BELGRADE     1092         0
BUDAPEST      214         0
DEBILT         82         0
DUSSELDORF     29         0
HEATHROW       82         0
KASSEL         11         0
LJUBLJANA      61         0
MAASTRICHT      9         0
MADRID        458         0
MUNCHENB        8         0
OSLO            5         0
STOCKHOLM       4         0
VALENTIA        1         0
